# P3Alpha

### Import libraries

In [3]:
import pandas as pd
import numpy as np
from scipy import sparse
import os
import csv

from utilities import load_sparse_matrix

### Load dataset

In [4]:
ICM = load_sparse_matrix("icm.npz")
URM = load_sparse_matrix("urm.npz")
URM_test = load_sparse_matrix("urm_test.npz")
URM_train = load_sparse_matrix("urm_train.npz")
URM_val = load_sparse_matrix("urm_val.npz")
URM_train_val = load_sparse_matrix("urm_train_val.npz")

Loaded icm.npz
Loaded urm.npz
Loaded urm_test.npz
Loaded urm_train.npz
Loaded urm_val.npz
Loaded urm_train_val.npz


In [5]:
from src.Evaluation.Evaluator import EvaluatorHoldout

evaluator_val = EvaluatorHoldout(URM_val, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 759 ( 1.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 322 ( 0.8%) Users that have less than 1 test interactions


In [6]:
from skopt.space import Real, Integer, Categorical

hyperparameters_range_dictionary = {
    "topK": Integer(5, 1000),
    "alpha": Real(0, 1),
}

In [7]:
from src.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from src.HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

recommender_class = P3alphaRecommender

hyperparameterSearch = SearchBayesianSkopt(
    recommender_class,
    evaluator_validation=evaluator_val,
    evaluator_test=evaluator_test)

In [19]:
from src.HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
  
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {},
    EARLYSTOPPING_KEYWORD_ARGS = {},
)

recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train_val],     # For a CBF model simply put [URM_train_validation, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {},
    EARLYSTOPPING_KEYWORD_ARGS = {},
)

In [9]:
import os

output_folder_path = "tuner/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 100
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

In [21]:
hyperparameterSearch.search(
    recommender_input_args,
    recommender_input_args_last_test = recommender_input_args_last_test,
    hyperparameter_search_space = hyperparameters_range_dictionary,
    n_cases = n_cases,
    n_random_starts = n_random_starts,
    save_model = "last",
    output_folder_path = output_folder_path, # Where to save the results
    output_file_name_root = recommender_class.RECOMMENDER_NAME, # How to call the files
    metric_to_optimize = metric_to_optimize,
    cutoff_to_optimize = cutoff_to_optimize,
)

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 902, 'alpha': 0.6800026623604563}
P3alphaRecommender: URM Detected 3461 (12.4%) items with no interactions.


KeyboardInterrupt: 

### Save recommendations

In [10]:
from utilities import save_recommendations

rec = P3alphaRecommender(URM)
rec.load_model(
    output_folder_path, 
    file_name=rec.RECOMMENDER_NAME + "_best_model_last.zip"
)
save_recommendations(rec)

P3alphaRecommender: URM Detected 3461 (12.4%) items with no interactions.
P3alphaRecommender: Loading model from file 'result_experiments/P3alphaRecommender_best_model_last.zip'
P3alphaRecommender: Loading complete
Saving recommendations finished!                                                                    
